In [1]:
import random

def generate_random_tree(n):
  parent = {}
  nodes = list(range(n))
  random.shuffle(nodes)
  root = nodes[0]
  parent[root] = root
  for i in range(1, len(nodes)):
    parent[nodes[i]] = nodes[random.randrange(i)]
  return parent

import time
def get_node_depths(tree):
    depths = {}  # size of tree, node:level
    #curr = root
    for node in tree:
        level = 0
        curr = node
        while tree[curr] != curr:
            if curr in depths:
                level += depths[curr]
                break  
            curr = tree[curr]
            level += 1
        depths[node] = level
    return depths

def get_distance(tree, src, dest, depths):
    s = src; d = dest
    while s != d:
        if depths[d] > depths[s]:
            d = tree[d]
        elif depths[s] > depths[d]:
            s = tree[s]
        else:
            if s != d and depths[s] == depths[d]:
                d = tree[d]
    lca = s
    src_depth = depths[src] - depths[lca]
    dest_depth = depths[dest] - depths[lca]
    return (src_depth + dest_depth)

# src, dst, distance_label
#
def generate_dataset(N):
    dataset = []     # node1, node2, distance
    node_id = {}
    id_to_node = {}
    node_pairs = {}  # (node1, node2): distance
    
    counter = 0
    trees = {}
    depths_graph = {}
    # give each node a unique id
    for n in range(3, N): # For each tree
        trees[n] = generate_random_tree(n)
        tree = trees[n]
        depths_graph[n] = get_node_depths(tree)        
        for node in tree: # For each node in the tree
            node_id[(n, node)] = counter
            id_to_node[counter] = n, node
            counter += 1
    
    for n in range(3, N):
        tree = trees[n]
        depths = depths_graph[n]
        for node in tree:
            for other_node in tree:
                dist = get_distance(tree, node, other_node, depths)
                dataset.append([node_id[(n, node)], node_id[(n, other_node)], dist])
                node_pairs[(node_id[(n, node)], node_id[(n, other_node)])] = dist
                
    return dataset, node_pairs, id_to_node, node_id
# generate_dataset(10)

In [2]:
from fastai.collab import *
from fastai.tabular.all import *

class CollabNN(Module):
    predictions = {}
    embeddings = {}
    def __init__(self, src_sz, y_range, n_act=200):
        self.node_factors = Embedding(*src_sz)
        self.layers = nn.Sequential(
            nn.Linear(src_sz[1]+src_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        self.prediction = None

    def forward(self, x):
        x_copy = torch.clone(x)
        embs = self.node_factors(x[:,0]), self.node_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        self.prediction = sigmoid_range(x, *self.y_range)

        for i in range(len(x)):
          pair = x_copy[i]
          CollabNN.predictions[(int(pair[0]),int(pair[1]))] = float(self.prediction[i])
          CollabNN.embeddings[int(pair[0])] = float(embs[0][i])
          CollabNN.embeddings[int(pair[1])] = float(embs[1][i])
        return self.prediction

    def get_predictions(self):
      return self.prediction

In [ ]:
import generator
import model
import pickle
import torch
from fastbook import *
from fastai.collab import *
from fastai.tabular.all import *
from fastai.losses import *
import argparse
import torch.nn as nn


def get_small_emb_sz(dls_df):
    emb = get_emb_sz(dls_df)
    return [(emb[0][0], 1)]


def MRELoss(inp, targ) -> Tensor:
    inp = torch.flatten(inp).float()
    targ = torch.flatten(targ).float()
    nom = torch.nn.functional.l1_loss(inp, targ)
    noo = torch.Tensor([0]).repeat(targ.size(0)).to(device)
    denom = torch.nn.functional.l1_loss(noo, targ)
    loss = (nom / denom).mean()
    return loss


def CombineLoss(inp, targ) -> Tensor:
    mse_loss = MSELossFlat()
    loss_1 = mse_loss(inp, targ)

    loss_2 = MRELoss(inp, targ)

    return alpha * loss_1 + (1 - alpha) * loss_2


lg_N = 10
num_epochs = 1000
alpha = 0.5
loss_function = CombineLoss


'''
Data can either be generated from scratch or read from a file that it was previously saved to
'''

savepath = 'save/' + str(alpha) + '_' + str(lg_N) + '_' + str(num_epochs) + '/'

dataset, node_pairs, id_to_node, node_id = generate_dataset(2**lg_N)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = np.array(dataset)

# if loading from csv, cut the first column out (line numbers)
# df = pd.DataFrame(data[:, 1:], columns=['src', 'dst', 'label'])
# node_pairs = pd.read_pickle(savepath + 'node_pairs.pkl')
# node_id = pd.read_pickle(savepath + 'node_id.pkl')

# otherwise (if generated), use data directly
df = pd.DataFrame(data, columns = ['src','dst','label'])


dls_df = CollabDataLoaders.from_df(df, bs=64)
embs = get_small_emb_sz(dls_df)
trainer = model.CollabNN(*embs, y_range=(0, embs[0][0]))
# learn = Learner(dls_df, trainer, loss_func = loss_function, path=savepath)

learn = Learner(dls_df, trainer, loss_func=loss_function, path=savepath, metrics=[mse, mae, MRELoss])

# # torch.save(trainer, modelpath + 'model.pth')
# learn.save('model')
# with open(savepath + "predictions.pkl", "wb") as outfile:
#     pickle.dump(model.CollabNN.predictions, outfile)
# with open(savepath + "embeddings.pkl", "wb") as outfile:
#     pickle.dump(model.CollabNN.embeddings, outfile)

# learn.load('model')
# SaveModelCallback(with_opt=True), 
learn.fit_one_cycle(n_epoch=num_epochs, lr_max=5e-3, wd=0.01, cbs=[ShowGraphCallback()])


In [ ]:
learn.recorder()